In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score

from geopy.distance import geodesic

In [ ]:
# Carga el dataset en un dataframe
df = pd.read_csv('uber_fares.csv')

# Nos quedamos con las filas cuya variable objetivo es un valor posible
df = df[df['fare_amount'] > 0]

# Separa el 80% para train y 20% para test
train, test, y_train, y_test = train_test_split(df.drop(columns=['fare_amount']), df['fare_amount'], test_size=0.2, random_state=1)

train.info()

In [ ]:
# Asigna el tipo de datos correcto a las variables que representan fechas
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'])
train['date'] = pd.to_datetime(train['date'])

### Análisis de valores nulos
Encontramos un valor nulo de *dropoff_location* y otro de *dropoff_latitude*, veamos las filas donde ocurren:

In [ ]:
train[train.isnull().any(axis=1)]

Ambos valores nulos se dan en la misma fila por lo que no podemos imputar la ubicación de destino ni estimar ninguna distancia recorrida, por lo tanto siendo que es una unica fila que representa menos del 0.0006% de los datos, procedemos a eliminarla.

In [ ]:
train = train.dropna()

In [ ]:
print(train['pickup_latitude'].describe())
print(train['dropoff_latitude'].describe())

In [ ]:
def coordenadas_absurdas(viaje):
    yield not -90 <= viaje['pickup_latitude'] <= 90, 'pickup_latitude'
    yield not -180 <= viaje['pickup_longitude'] <= 180, 'pickup_longitude'
    yield not -90 <= viaje['dropoff_latitude'] <= 90, 'dropoff_latitude'
    yield not -180 <= viaje['dropoff_longitude'] <= 180 , 'dropoff_longitude'


def imputar_con_knn(viaje, coordenada):
    #TODO: implementar KNN
    return 0

def procesar_coordenadas_absurdas(df, imputar=True):
    def procesar_fila(viaje):
        coordenada = None
        for absurda, coord in coordenadas_absurdas(viaje):
            if absurda:
                if coordenada or imputar == False:  # Más de una absurda o no se quiere imputar
                    return None
                coordenada = coord

        if coordenada:
            viaje[coordenada] = imputar_con_knn(viaje, coordenada)
        return viaje

    
    df = df.apply(procesar_fila, axis=1)
    df = df.dropna()

    return df

train = procesar_coordenadas_absurdas(train)
test = procesar_coordenadas_absurdas(test, imputar=False)